# First steps with Linkerd 2.0
In this notebook, we explore the service mesh Linkerd 2.0.

References:
* https://kubernetes.io/blog/2018/09/18/hands-on-with-linkerd-2.0/
* https://linkerd.io/2/getting-started/

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Set-up-a-local-single-node-Kubernetes-cluster" data-toc-modified-id="Set-up-a-local-single-node-Kubernetes-cluster-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Set up a local single-node Kubernetes cluster</a></span></li><li><span><a href="#Set-up-a-demo-application-in-the-Kubernetes-cluster" data-toc-modified-id="Set-up-a-demo-application-in-the-Kubernetes-cluster-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Set up a demo application in the Kubernetes cluster</a></span><ul class="toc-item"><li><span><a href="#Deploy-the-demo-app-emojivoto" data-toc-modified-id="Deploy-the-demo-app-emojivoto-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Deploy the demo app <em>emojivoto</em></a></span></li><li><span><a href="#Find-out-on-which-port-the-service-is-running" data-toc-modified-id="Find-out-on-which-port-the-service-is-running-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Find out on which port the service is running</a></span></li><li><span><a href="#Map-the-port-to-a-port-on-the-host-system-with-`kubectl-port-forward'" data-toc-modified-id="Map-the-port-to-a-port-on-the-host-system-with-`kubectl-port-forward'-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Map the port to a port on the host system with `kubectl port-forward'</a></span></li></ul></li><li><span><a href="#Set-up-Linkderd-2" data-toc-modified-id="Set-up-Linkderd-2-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Set up Linkderd 2</a></span><ul class="toc-item"><li><span><a href="#Install-Linkerd-2-in-the-user's-home-directory" data-toc-modified-id="Install-Linkerd-2-in-the-user's-home-directory-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Install Linkerd 2 in the user's home directory</a></span></li><li><span><a href="#Deploy-Linkerd’s-control-plane-to-the-cluster" data-toc-modified-id="Deploy-Linkerd’s-control-plane-to-the-cluster-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Deploy Linkerd’s control plane to the cluster</a></span></li><li><span><a href="#Add-Linkerd-to-the-web-service" data-toc-modified-id="Add-Linkerd-to-the-web-service-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Add Linkerd to the web service</a></span></li></ul></li><li><span><a href="#Shut-down-the-Kubernetes-cluster" data-toc-modified-id="Shut-down-the-Kubernetes-cluster-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Shut down the Kubernetes cluster</a></span></li></ul></div>

## Set up a local single-node Kubernetes cluster
To see how this works, look at [../2019-09-16-setting-up-k8s-cluster-with-kind/setting-up-k8s-cluster-with-kind.ipynb](setting-up-k8s-cluster-with-kind.ipynb)

In [1]:
. kubernetes-utils
create-kind-cluster linkerd-test

Creating cluster "linkerd-test" ...
 ✓ Ensuring node image (kindest/node:v1.15.3) 🖼 
 ✓ Preparing nodes 📦 
 ✓ Creating kubeadm config 📜 
 ✓ Starting control-plane 🕹️ 
 ✓ Installing CNI 🔌 
 ✓ Installing StorageClass 💾 
Cluster creation complete. You can now use the cluster with:

export KUBECONFIG="$(kind get kubeconfig-path --name="linkerd-test")"
kubectl cluster-info


## Set up a demo application in the Kubernetes cluster

### Deploy the demo app *emojivoto*
Note that we use `kubectl rollout status` to wait until the deployments are finished. Unfortunately, we cannot pass `emojivoto.yml` to this command directly because it only supports files which contain exactly one deployment, see, e.g., https://github.com/kubernetes/kubernetes/issues/72216

In [2]:
export EMOJIVOTO=download/emojivoto.yml

if [ ! -f $EMOJIVOTO ]; then
    mkdir -p download
    curl -s https://run.linkerd.io/emojivoto.yml -o $EMOJIVOTO
fi

kubectl apply -f $EMOJIVOTO

kubectl rollout status -n emojivoto deployment emoji
kubectl rollout status -n emojivoto deployment voting
kubectl rollout status -n emojivoto deployment web
kubectl rollout status -n emojivoto deployment vote-bot

namespace/emojivoto created
serviceaccount/emoji created
serviceaccount/voting created
serviceaccount/web created
service/emoji-svc created
service/voting-svc created
service/web-svc created
deployment.apps/emoji created
deployment.apps/vote-bot created
deployment.apps/voting created
deployment.apps/web created
Waiting for deployment spec update to be observed...
Waiting for deployment spec update to be observed...
Waiting for deployment "emoji" rollout to finish: 0 out of 1 new replicas have been updated...
Waiting for deployment "emoji" rollout to finish: 0 of 1 updated replicas are available...
deployment "emoji" successfully rolled out
deployment "voting" successfully rolled out
deployment "web" successfully rolled out
deployment "vote-bot" successfully rolled out


### Find out on which port the service is running

In [3]:
kubectl get all -n emojivoto

NAME                           READY   STATUS    RESTARTS   AGE
pod/emoji-89d7ff8b9-8rb7k      1/1     Running   0          69s
pod/vote-bot-56c4b4cdb-c9tvx   1/1     Running   0          69s
pod/voting-746477d54b-cm4fh    1/1     Running   0          69s
pod/web-5f96ddbd6b-q88hz       1/1     Running   0          69s

NAME                 TYPE        CLUSTER-IP       EXTERNAL-IP   PORT(S)             AGE
service/emoji-svc    ClusterIP   10.98.95.104     <none>        8080/TCP,8801/TCP   72s
service/voting-svc   ClusterIP   10.98.226.227    <none>        8080/TCP,8801/TCP   72s
service/web-svc      ClusterIP   10.100.139.120   <none>        80/TCP              72s

NAME                       READY   UP-TO-DATE   AVAILABLE   AGE
deployment.apps/emoji      1/1     1            1           72s
deployment.apps/vote-bot   1/1     1            1           72s
deployment.apps/voting     1/1     1            1           72s
deployment.apps/web        1/1     1            1           72s

NAME 

In [4]:
kubectl get -n emojivoto services

NAME         TYPE        CLUSTER-IP       EXTERNAL-IP   PORT(S)             AGE
emoji-svc    ClusterIP   10.98.95.104     <none>        8080/TCP,8801/TCP   73s
voting-svc   ClusterIP   10.98.226.227    <none>        8080/TCP,8801/TCP   73s
web-svc      ClusterIP   10.100.139.120   <none>        80/TCP              73s


In [5]:
# Note that kubectl port-forward needs the service port *inside the container*
EMOJIVOTO_WEB_PORT=$(kubectl get -n emojivoto service web-svc -o jsonpath='{.spec.ports[0].port}')
echo "The port of the service web-svc is $EMOJIVOTO_WEB_PORT"

The port of the service web-svc is 80


### Map the port to a port on the host system with `kubectl port-forward'

In [6]:
# Just choose a port which is hopefully unused on the local system
LOCAL_WEB_PORT=8097
echo "Setting up access to the emoji vote service at http://127.0.0.1:$LOCAL_WEB_PORT"
kubectl port-forward -n emojivoto service/web-svc $LOCAL_WEB_PORT:$EMOJIVOTO_WEB_PORT > /dev/null &

Setting up access to the emoji vote service at http://127.0.0.1:8097
[1] 8403


## Set up Linkderd 2

### Install Linkerd 2 in the user's home directory

In [7]:
if [ ! -d $HOME/.linkerd2 ]; then
    curl -sL https://run.linkerd.io/install | sh
    echo
fi

export PATH=$PATH:$HOME/.linkerd2/bin

linkerd version

Client version: stable-2.9.3
Server version: unavailable


### Deploy Linkerd’s control plane to the cluster

In [8]:
linkerd check --pre

kubernetes-api
--------------
√ can initialize the client
√ can query the Kubernetes API

kubernetes-version
------------------
√ is running the minimum Kubernetes API version
√ is running the minimum kubectl version

pre-kubernetes-setup
--------------------
√ control plane namespace does not already exist
√ can create non-namespaced resources
√ can create ServiceAccounts
√ can create Services
√ can create Deployments
√ can create CronJobs
√ can create ConfigMaps
√ can create Secrets
√ can read Secrets
√ can read extension-apiserver-authentication configmap
√ no clock skew detected

pre-kubernetes-capability
-------------------------
√ has NET_ADMIN capability
√ has NET_RAW capability

linkerd-version
---------------
√ can determine the latest version
‼ cli is up-to-date
    is running version 2.9.3 but the latest stable version is 2.9.4
    see https://linkerd.io/checks/#l5d-version-cli for hints

Status check results are √


In [9]:
linkerd install | kubectl apply -f -

kubectl rollout status -n linkerd deploy linkerd-identity
kubectl rollout status -n linkerd deploy linkerd-controller
kubectl rollout status -n linkerd deploy linkerd-web
kubectl rollout status -n linkerd deploy linkerd-prometheus
kubectl rollout status -n linkerd deploy linkerd-grafana
kubectl rollout status -n linkerd deploy linkerd-proxy-injector
kubectl rollout status -n linkerd deploy linkerd-sp-validator
kubectl rollout status -n linkerd deploy linkerd-tap

namespace/linkerd created
clusterrole.rbac.authorization.k8s.io/linkerd-linkerd-identity created
clusterrolebinding.rbac.authorization.k8s.io/linkerd-linkerd-identity created
serviceaccount/linkerd-identity created
clusterrole.rbac.authorization.k8s.io/linkerd-linkerd-controller created
clusterrolebinding.rbac.authorization.k8s.io/linkerd-linkerd-controller created
serviceaccount/linkerd-controller created
clusterrole.rbac.authorization.k8s.io/linkerd-linkerd-destination created
clusterrolebinding.rbac.authorization.k8s.io/linkerd-linkerd-destination created
serviceaccount/linkerd-destination created
role.rbac.authorization.k8s.io/linkerd-heartbeat created
rolebinding.rbac.authorization.k8s.io/linkerd-heartbeat created
serviceaccount/linkerd-heartbeat created
role.rbac.authorization.k8s.io/linkerd-web created
rolebinding.rbac.authorization.k8s.io/linkerd-web created
clusterrole.rbac.authorization.k8s.io/linkerd-linkerd-web-check created
clusterrolebinding.rbac.authorization.k8s.io/link

In [10]:
linkerd check

kubernetes-api
--------------
√ can initialize the client
√ can query the Kubernetes API

kubernetes-version
------------------
√ is running the minimum Kubernetes API version
√ is running the minimum kubectl version

linkerd-existence
-----------------
√ 'linkerd-config' config map exists
√ heartbeat ServiceAccount exist
√ control plane replica sets are ready
√ no unschedulable pods
√ controller pod is running
√ can initialize the client
√ can query the control plane API

linkerd-config
--------------
√ control plane Namespace exists
√ control plane ClusterRoles exist
√ control plane ClusterRoleBindings exist
√ control plane ServiceAccounts exist
√ control plane CustomResourceDefinitions exist
√ control plane MutatingWebhookConfigurations exist
√ control plane ValidatingWebhookConfigurations exist
√ control plane PodSecurityPolicies exist

linkerd-identity
----------------
√ certificate config is valid
√ trust anchors are using supported crypto algorithm
√ trust anchors are within the

In [11]:
linkerd dashboard &
sleep 1 # make sure that the initial output of 'linkerd dashboard' is output below the cell

[2] 14292
Linkerd dashboard available at:
http://localhost:50750
Grafana dashboard available at:
http://localhost:50750/grafana
Opening Linkerd dashboard in the default browser


### Add Linkerd to the web service

In [12]:
kubectl get -n emojivoto deploy/web -o yaml | linkerd inject - | kubectl apply -f -


deployment "web" injected

deployment.extensions/web configured


Now, have a look at the Linkerd dashboard and see what information about the service is gathered.

## Shut down the Kubernetes cluster

In [13]:
delete-kind-cluster

Deleting cluster "linkerd-test" ...
E0223 23:17:00.335272   14292 portforward.go:233] lost connection to pod
E0223 23:17:00.335240   14292 portforward.go:233] lost connection to pod
E0223 23:17:00.336420    8403 portforward.go:233] lost connection to pod
[1]-  Fertig                  kubectl port-forward -n emojivoto service/web-svc $LOCAL_WEB_PORT:$EMOJIVOTO_WEB_PORT > /dev/null
